In [3]:
import mlflow
# Step 2: Set up the MLflow tracking server
mlflow.set_tracking_uri("http://ec2-13-60-32-83.eu-north-1.compute.amazonaws.com:5000/")

In [4]:
# Set or create an experiment
mlflow.set_experiment("ML Algos with HP Tuning")

<Experiment: artifact_location='s3://mlflow-yt-comment-analyzer-bucket/627515249629414087', creation_time=1730017653834, experiment_id='627515249629414087', last_update_time=1730017653834, lifecycle_stage='active', name='ML Algos with HP Tuning', tags={}>

In [5]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE
import mlflow
import mlflow.sklearn
import optuna

In [6]:
df = pd.read_csv("/media/ubaid-ur-rehman/Artifacts/CodeData/jupyter notebook/End to End  ML Projects/yt-comment-analyzer-preprocessing/reddit_preprocessing.csv")
display(df.shape,df.dropna(inplace=True)
,df.shape)

(36799, 2)

None

(36662, 2)

In [7]:
# Step 1: (Optional) Remapping - skipped since not strictly needed for SVM

# Step 2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

# Step 3: TF-IDF vectorizer setup
ngram_range = (1, 3)  # Trigram
max_features = 1000  # Set max_features to 1000
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X = vectorizer.fit_transform(df['clean_comment'])
y = df['category']

# Step 4: Apply SMOTE to handle class imbalance
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Step 5: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)

# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")


# Step 6: Optuna objective function for SVM
def objective_svm(trial):
    C = trial.suggest_float('C', 1e-4, 10.0, log=True)
    kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly'])

    model = SVC(C=C, kernel=kernel, random_state=42)
    return accuracy_score(y_test, model.fit(X_train, y_train).predict(X_test))


# Step 7: Run Optuna for SVM, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_svm, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = SVC(C=best_params['C'], kernel=best_params['kernel'], random_state=42)

    # Log the best model with MLflow, passing the algo_name as "SVM"
    log_mlflow("SVM", best_model, X_train, X_test, y_train, y_test)

# Run the experiment for SVM
run_optuna_experiment()


[I 2024-10-27 13:39:54,979] A new study created in memory with name: no-name-9739ffb8-7948-4cb6-93ad-4b27afbbacdc
[I 2024-10-27 13:42:27,430] Trial 0 finished with value: 0.543859649122807 and parameters: {'C': 0.07856141419917174, 'kernel': 'poly'}. Best is trial 0 with value: 0.543859649122807.
[I 2024-10-27 13:54:25,239] Trial 1 finished with value: 0.6955189177763687 and parameters: {'C': 2.9070849343221847, 'kernel': 'poly'}. Best is trial 1 with value: 0.6955189177763687.
[I 2024-10-27 13:59:45,938] Trial 2 finished with value: 0.3628196998520397 and parameters: {'C': 0.0008503090770892965, 'kernel': 'poly'}. Best is trial 1 with value: 0.6955189177763687.
[I 2024-10-27 14:03:17,926] Trial 3 finished with value: 0.7300782075671105 and parameters: {'C': 0.08263701165544042, 'kernel': 'linear'}. Best is trial 3 with value: 0.7300782075671105.
[I 2024-10-27 14:07:19,798] Trial 4 finished with value: 0.7210949059395476 and parameters: {'C': 0.06308074914450243, 'kernel': 'linear'}. B

In [8]:
.

SyntaxError: invalid syntax (1933637684.py, line 1)